In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import pickle
import json
import random
import music_tag

from music_utils import *

from pathlib import Path
import numpy as np

from extract_openl3_embeddings import EmbeddingsOpenL3
from essentia.standard import MonoLoader, TensorflowPredictEffnetDiscogs, TensorflowPredict2D

import tensorflow as tf
tf.config.run_functions_eagerly(True)

os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
   print("No GPU found. Please ensure you have installed TensorFlow correctly")
print("Num GPUs:", len(physical_devices))

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


Num GPUs: 1


In [2]:
DATASET = Path("Music_Dataset")
CLASSES = "mtg_jamendo_genre.json"

embedding_87_model = TensorflowPredictEffnetDiscogs(graphFilename="discogs-effnet-bs64-1.pb", output="PartitionedCall:1")
prediction_87_model = TensorflowPredict2D(graphFilename="mtg_jamendo_genre-discogs-effnet-1.pb")

embedding_512_model = EmbeddingsOpenL3("openl3-music-mel128-emb512-3.pb")

[   INFO   ] TensorflowPredict: Successfully loaded graph file: `discogs-effnet-bs64-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `mtg_jamendo_genre-discogs-effnet-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `openl3-music-mel128-emb512-3.pb`


In [8]:
random_mp3 = pick_random_mp3(DATASET)
if check_predictions_87(random_mp3):
    print(random_mp3[8:], get_top_5_genres(random_mp3, CLASSES))
else:
    print("No predictions for\n")
    print_info(random_mp3)

No predictions for

filename: 14 Stack-O-Lee.mp3
filepath: Music_Dataset/Black_Snake_Moan_OST/14 Stack-O-Lee.mp3
folder: Music_Dataset/Black_Snake_Moan_OST
filesize: 8.03
title: Stack-O-Lee
artist: Samuel L. Jackson
album: Black snake moan
year: 2007
tracknumber: 14
genre: Soundtrack
predictions_87: []
embedding_512: []


In [9]:
def extract_predictions_87(mp3_file_path, embedding_model, prediction_model):
    try:
        audio = MonoLoader(filename=mp3_file_path, sampleRate=16000, resampleQuality=2)()
        embeddings = embedding_model(audio)
        predictions = prediction_model(embeddings)
        averaged_predictions = np.mean(predictions, axis=0)
        return averaged_predictions
    except Exception as e:
        print(f"Error extracting predictions from {mp3_file_path}: {e}")
        return None

In [10]:
def extract_embeddings_512(file_path, extractor):
    try:
        vector = extractor.compute(file_path)
        embedding = vector.mean(axis=0)
        return embedding
    except Exception as e:
        print(f"Error extracting embeddingss from {file_path}: {e}")
        return None

In [12]:
# This function iterates over all mp3 files in a dataset, 
# checks and updates predictions and embeddings in their corresponding pickle files.

total_files = len(list(DATASET.rglob('*.mp3')))
valid_files = 0

for counter, mp3_file in enumerate(DATASET.rglob('*.mp3'), start=1):
    pkl_path = mp3_file.with_suffix('.pkl')
    
    try:
        with pkl_path.open('rb') as f:
            file_info = pickle.load(f)
    except Exception as e:
        print(f"Error loading pickle file {pkl_path}: {e}")
        continue

    if check_predictions_87(mp3_file) and check_embeddings_512(mp3_file):
        valid_files += 1
        continue

    try:
        if not check_predictions_87(mp3_file):
            file_info["predictions_87"] = extract_predictions_87(str(mp3_file), embedding_87_model, prediction_87_model)
    except Exception as e:
        print(f"Error extracting predictions for {mp3_file}: {e}")
        continue

    try:
        if not check_embeddings_512(mp3_file):
            file_info["embedding_512"] = extract_embeddings_512(str(mp3_file), embedding_512_model)
    except Exception as e:
        print(f"Error extracting embeddings for {mp3_file}: {e}")
        continue

    try:
        with pkl_path.open('wb') as f:
            pickle.dump(file_info, f)
    except Exception as e:
        print(f"Error saving pickle file {pkl_path}: {e}")
        continue

    # Print progress every 2.5% of the total valid files
    if counter % (total_files // 40) == 0:
        print(f"Processed {counter} files")


print(f"{valid_files} valid files out of {total_files} total files")

#582 files in 66min so around 22h for 11645 files
#1164 files in 133min so around 22h for 11645 files

Processed 4 files
Processed 8 files
Processed 12 files
Processed 16 files
Processed 20 files
Processed 24 files
Processed 28 files
Processed 32 files
Processed 36 files
Processed 40 files
Processed 44 files
Processed 48 files
Processed 52 files
Processed 56 files
Processed 60 files
Processed 64 files
Processed 68 files
Processed 72 files
Processed 76 files
Processed 80 files
Processed 84 files
Processed 88 files
Processed 92 files
Processed 96 files
Processed 100 files
Processed 104 files
Processed 108 files
Processed 112 files
Processed 116 files
Processed 120 files
Processed 124 files
Processed 128 files
Processed 132 files
Processed 136 files
Processed 140 files
Processed 144 files
Processed 148 files
Processed 152 files
Processed 156 files
Processed 160 files
Processed 164 files
Processed 168 files
Processed 172 files
0 valid files out of 174 total files


In [ ]:
# random_file = pick_random_mp3(DATASET)

# if check_embeddings_512(random_file) and check_predictions_87(random_file):
#     print("embeddings and predictions are valid\n")

# print_info(random_file)
